Importa bibliotecas para trabalhar com o NLTK (Natural Language Toolkit)

In [8]:
from nltk.util import pad_sequence
from nltk.util import bigrams
from nltk.util import ngrams
from nltk.util import everygrams
from nltk.lm import Laplace
from nltk.lm import KneserNeyInterpolated
from nltk.lm.preprocessing import pad_both_ends
from nltk.lm.preprocessing import flatten
from nltk.lm.preprocessing import padded_everygram_pipeline
from nltk import word_tokenize, sent_tokenize
import string

In [9]:
import nltk
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

Para gerar bigramas de um texto, primeiro é necessário transformar as palavras em tokens.

Entretanto, o primeiro passo é separar as frases do texto. A função `sent_tokenize` utiliza um tokenizador de frases, que separa as frases em uma lista conforme a pontuação presente no texto.

Com as frases separadas, é necessário separar as palavras do texto para que elas seja transformadas em tokens. Para isso, é utilizado o `word_tokenize` que separa as palavras no texto, conforme a ocorrência de espaços, virgulas, entre outros.

Aqui, `tokenized_text` contém o texto tokeniado por frases encontradas no texto

In [10]:
text = 'The quick brown fox jumps over the lazy dog. The five boxing wizards jump quickly.'
sentences = sent_tokenize(text)
print(sentences)
print(word_tokenize(sentences[0]))
tokenized_text = [list(map(str.lower, word_tokenize(sent)))
                  for sent in sent_tokenize(text)]
print(tokenized_text)


['The quick brown fox jumps over the lazy dog.', 'The five boxing wizards jump quickly.']
['The', 'quick', 'brown', 'fox', 'jumps', 'over', 'the', 'lazy', 'dog', '.']
[['the', 'quick', 'brown', 'fox', 'jumps', 'over', 'the', 'lazy', 'dog', '.'], ['the', 'five', 'boxing', 'wizards', 'jump', 'quickly', '.']]


A função `padded_everygram_pipeline` gera, a partir dos tokens, n-gramas conforme parâmetro e também uma lista com os tokens do texto arrajados conforme a sua disposição no texto.

Observe que essa função já adiciona os símbolos terminadores de frase, entretanto o ponto final foi transformado em uma palavra por estar contido no texto.

Para evitar de gerar esse tipo de token, a pontuação deve ser removida da lista de tokens

In [19]:
n = 2
train_data, vocab = padded_everygram_pipeline(n, tokenized_text)
# Filter for bigrams before printing
print([ngram for ngram in list(flatten(train_data)) if len(ngram) == 2])
print(list(vocab))

[('<s>', 'the'), ('the', 'quick'), ('quick', 'brown'), ('brown', 'fox'), ('fox', 'jumps'), ('jumps', 'over'), ('over', 'the'), ('the', 'lazy'), ('lazy', 'dog'), ('dog', '.'), ('.', '</s>'), ('<s>', 'the'), ('the', 'five'), ('five', 'boxing'), ('boxing', 'wizards'), ('wizards', 'jump'), ('jump', 'quickly'), ('quickly', '.'), ('.', '</s>')]
['<s>', 'the', 'quick', 'brown', 'fox', 'jumps', 'over', 'the', 'lazy', 'dog', '.', '</s>', '<s>', 'the', 'five', 'boxing', 'wizards', 'jump', 'quickly', '.', '</s>']


Aqui é removido da lista todas as palavras que sejam iguais a lista d pontuação da classe string.

Então, a função `padded_everygram_pipeline` é processada novamente. Agora, os sinais de pontuação foram removidos.

In [12]:
# Remove punctuation tokens
filtered_word_tokens_from_sentences = [[word for word in sentence if word not in string.punctuation]
                                         for sentence in tokenized_text]

print(filtered_word_tokens_from_sentences)

# Preprocess the tokenized text for 3-grams language modelling
n = 2
train_data, vocab = padded_everygram_pipeline(n, filtered_word_tokens_from_sentences)
print(list(flatten(train_data)))
print(list(vocab))

[['the', 'quick', 'brown', 'fox', 'jumps', 'over', 'the', 'lazy', 'dog'], ['the', 'five', 'boxing', 'wizards', 'jump', 'quickly']]
[('<s>',), ('<s>', 'the'), ('the',), ('the', 'quick'), ('quick',), ('quick', 'brown'), ('brown',), ('brown', 'fox'), ('fox',), ('fox', 'jumps'), ('jumps',), ('jumps', 'over'), ('over',), ('over', 'the'), ('the',), ('the', 'lazy'), ('lazy',), ('lazy', 'dog'), ('dog',), ('dog', '</s>'), ('</s>',), ('<s>',), ('<s>', 'the'), ('the',), ('the', 'five'), ('five',), ('five', 'boxing'), ('boxing',), ('boxing', 'wizards'), ('wizards',), ('wizards', 'jump'), ('jump',), ('jump', 'quickly'), ('quickly',), ('quickly', '</s>'), ('</s>',)]
['<s>', 'the', 'quick', 'brown', 'fox', 'jumps', 'over', 'the', 'lazy', 'dog', '</s>', '<s>', 'the', 'five', 'boxing', 'wizards', 'jump', 'quickly', '</s>']


É criado o modelo do tipo KneserNeyInterpolated. Os dados de `train_data` e `vocab`são processador novamente pois as funções de print na célula anterior alteram o estado desses objetos

In [13]:
model = KneserNeyInterpolated(n)
train_data, vocab = padded_everygram_pipeline(n, filtered_word_tokens_from_sentences)

É realizado o treinamento do modelo. Após exibe-se as informações do vocabulário gerado.

In [14]:
model.fit(train_data, vocab)
print(model.vocab)

<Vocabulary with cutoff=1 unk_label='<UNK>' and 16 items>


Apresenta a pontuação de cada *token* do vocabulário gerado

In [15]:
for word in model.vocab:
  print(f"Score for {word}: {model.score(word)}")

Score for <s>: 0.0
Score for the: 0.125
Score for quick: 0.0625
Score for brown: 0.0625
Score for fox: 0.0625
Score for jumps: 0.0625
Score for over: 0.0625
Score for lazy: 0.0625
Score for dog: 0.0625
Score for </s>: 0.125
Score for five: 0.0625
Score for boxing: 0.0625
Score for wizards: 0.0625
Score for jump: 0.0625
Score for quickly: 0.0625
Score for <UNK>: 0.0


In [16]:

print(model.vocab.lookup('the quick brown fox lah .'.split()))

('the', 'quick', 'brown', 'fox', '<UNK>', '<UNK>')


In [17]:
print(model.counts)

<NgramCounter with 2 ngram orders and 36 ngrams>
